# Fractional Translation Optimization

Implement fractional translation from top kernel to improve solutions.
Use the CORRECT baseline: snapshot 21337353543 (score 70.615107, 0 overlaps)

In [ ]:
import os
import json
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
import time

# Tree polygon vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree(x, y, angle):
    poly = Polygon(zip(TX, TY))
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def calculate_side(trees):
    polys = [create_tree(*t) for t in trees]
    union = unary_union(polys)
    bounds = union.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def calculate_score_for_n(trees, n):
    side = calculate_side(trees)
    return (side ** 2) / n

def has_overlap(trees):
    polys = [create_tree(*t) for t in trees]
    for i in range(len(polys)):
        for j in range(i+1, len(polys)):
            if polys[i].intersects(polys[j]) and not polys[i].touches(polys[j]):
                return True
    return False

print("Functions defined!")

In [ ]:
# Load the CORRECT baseline (21337353543)
VALID_BASELINE = '/home/nonroot/snapshots/santa-2025/21337353543/submission/submission.csv'

def load_baseline(path):
    """Load baseline preserving original string precision."""
    rows_by_n = {}
    trees_by_n = {}
    with open(path, 'r') as f:
        next(f)  # Skip header
        for line in f:
            parts = line.strip().split(',')
            if len(parts) != 4:
                continue
            id_val, x_str, y_str, deg_str = parts
            n = int(id_val.split('_')[0])
            
            # Parse coordinates
            x = float(x_str[1:] if x_str.startswith('s') else x_str)
            y = float(y_str[1:] if y_str.startswith('s') else y_str)
            angle = float(deg_str[1:] if deg_str.startswith('s') else deg_str)
            
            if n not in rows_by_n:
                rows_by_n[n] = []
                trees_by_n[n] = []
            rows_by_n[n].append(parts)
            trees_by_n[n].append((x, y, angle))
    
    return rows_by_n, trees_by_n

baseline_rows, baseline_trees = load_baseline(VALID_BASELINE)

# Calculate baseline scores
baseline_scores = {n: calculate_score_for_n(baseline_trees[n], n) for n in range(1, 201)}
baseline_total = sum(baseline_scores.values())
print(f"Baseline total score: {baseline_total:.6f}")

In [ ]:
# Verify baseline has no overlaps
print("Verifying baseline has no overlaps...")
overlap_count = 0
for n in range(1, 201):
    if has_overlap(baseline_trees[n]):
        overlap_count += 1
        if overlap_count <= 5:
            print(f"  N={n} has overlaps!")

if overlap_count == 0:
    print("✅ Baseline has NO overlaps - this is the correct baseline!")
else:
    print(f"❌ Baseline has {overlap_count} N values with overlaps!")

In [ ]:
# Implement fractional translation
def fractional_translation(trees, n, step_sizes=[0.001, 0.0005, 0.0002, 0.0001], max_iterations=100):
    """Improve solution by small translations in 8 directions."""
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
    best_side = calculate_side(trees)
    best_trees = list(trees)
    
    for iteration in range(max_iterations):
        improved = False
        for i in range(len(best_trees)):
            for step in step_sizes:
                for dx, dy in directions:
                    new_trees = list(best_trees)
                    x, y, angle = new_trees[i]
                    new_trees[i] = (x + dx * step, y + dy * step, angle)
                    
                    if not has_overlap(new_trees):
                        new_side = calculate_side(new_trees)
                        if new_side < best_side - 1e-10:
                            best_side = new_side
                            best_trees = new_trees
                            improved = True
        
        if not improved:
            break
    
    return best_trees, best_side

print("Fractional translation function defined!")

In [ ]:
# Test on small N values first
print("Testing fractional translation on small N values...")
print("="*60)

test_ns = [10, 15, 20, 25, 30]
test_results = {}

for n in test_ns:
    start_time = time.time()
    baseline_side = calculate_side(baseline_trees[n])
    baseline_score = calculate_score_for_n(baseline_trees[n], n)
    
    improved_trees, improved_side = fractional_translation(baseline_trees[n], n)
    improved_score = (improved_side ** 2) / n
    
    elapsed = time.time() - start_time
    improvement = baseline_score - improved_score
    
    test_results[n] = {
        'baseline_score': baseline_score,
        'improved_score': improved_score,
        'improvement': improvement,
        'time': elapsed
    }
    
    print(f"N={n}: baseline={baseline_score:.6f}, improved={improved_score:.6f}, gain={improvement:.6f} ({elapsed:.1f}s)")

print("\nTotal test improvement:", sum(r['improvement'] for r in test_results.values()))

In [ ]:
# If improvements found, run on more N values
# Focus on N=2-50 which contribute most to score
print("\nRunning fractional translation on N=2-50...")
print("="*60)

improved_solutions = {}
total_improvement = 0

for n in range(2, 51):  # Skip N=1 (already optimal)
    start_time = time.time()
    baseline_score = baseline_scores[n]
    
    improved_trees, improved_side = fractional_translation(baseline_trees[n], n)
    improved_score = (improved_side ** 2) / n
    
    # Validate no overlaps
    if has_overlap(improved_trees):
        print(f"N={n}: OVERLAP - keeping baseline")
        improved_solutions[n] = baseline_trees[n]
    else:
        improved_solutions[n] = improved_trees
        improvement = baseline_score - improved_score
        if improvement > 1e-10:
            total_improvement += improvement
            print(f"N={n}: improved by {improvement:.6f}")

print(f"\nTotal improvement for N=2-50: {total_improvement:.6f}")

In [ ]:
# Build final solution
# Use improved solutions where we found improvements, baseline elsewhere
final_solution = {}

for n in range(1, 201):
    if n in improved_solutions:
        final_solution[n] = improved_solutions[n]
    else:
        final_solution[n] = baseline_trees[n]

# Calculate final score
final_scores = {n: calculate_score_for_n(final_solution[n], n) for n in range(1, 201)}
final_total = sum(final_scores.values())

print(f"Baseline total: {baseline_total:.6f}")
print(f"Final total: {final_total:.6f}")
print(f"Total improvement: {baseline_total - final_total:.6f}")

In [ ]:
# Validate ALL solutions for overlaps before saving
print("\nValidating all solutions for overlaps...")
overlap_ns = []
for n in range(1, 201):
    if has_overlap(final_solution[n]):
        overlap_ns.append(n)
        # Fall back to baseline
        final_solution[n] = baseline_trees[n]

if overlap_ns:
    print(f"Found overlaps in {len(overlap_ns)} N values, fell back to baseline")
    # Recalculate score
    final_scores = {n: calculate_score_for_n(final_solution[n], n) for n in range(1, 201)}
    final_total = sum(final_scores.values())
    print(f"Adjusted final total: {final_total:.6f}")
else:
    print("✅ All solutions pass overlap validation!")

In [ ]:
# Save submission
def save_submission(solution, output_path):
    """Save solution to CSV with proper format."""
    with open(output_path, 'w') as f:
        f.write('id,x,y,deg\n')
        for n in range(1, 201):
            trees = solution[n]
            for idx, (x, y, angle) in enumerate(trees):
                f.write(f"{n:03d}_{idx},s{x},s{y},s{angle}\n")

save_submission(final_solution, '/home/submission/submission.csv')
print("Saved submission to /home/submission/submission.csv")

# Verify
with open('/home/submission/submission.csv', 'r') as f:
    lines = f.readlines()
print(f"Total lines: {len(lines)}")
print(f"First few lines:")
for line in lines[:5]:
    print(f"  {line.strip()}")

In [ ]:
# Save metrics
metrics = {
    'cv_score': final_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - final_total,
    'test_results': test_results,
    'overlap_fallbacks': len(overlap_ns) if overlap_ns else 0,
    'notes': 'Fractional translation optimization on valid baseline 21337353543'
}

with open('/home/code/experiments/007_fractional_translation/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2, default=str)

print("Metrics saved.")
print(f"CV Score: {final_total:.6f}")